<a href="https://colab.research.google.com/github/anujsahani01/English-Marathi-Translation/blob/main/Data_Preprocessing(translation).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install kaggle
!pip install datasets
!pip install --upgrade accelerate
!pip install wandb
!pip uninstall transformers
!pip install --no-cache-dir transformers sentencepiece
!pip install evaluate
!pip install rouge_score
!pip install sacremoses
!pip install sacrebleu
!pip install langid

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import langid
import pandas as pd
import datasets
import statistics

In [ ]:
from datasets import load_dataset
dataset = load_dataset("anujsahani01/English-Marathi")
dataset

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['english', 'marathi'],
        num_rows: 2637962
    })
    test: Dataset({
        features: ['english', 'marathi'],
        num_rows: 879321
    })
})

In [ ]:
def filter(data):
  data = data.replace('*', '').replace(':', '').replace(';', '').replace('$', '').replace('#', '').replace('@', '').replace('%', '').replace('&', '').replace('-', '').replace('_', '').replace('+', '').replace('~', '').replace('`', '').replace('>', '').replace('<', '').replace("|", '').replace('^', '').replace('(', '').replace(')', '').replace('{', '').replace('}', '').replace('!', '').replace('"', '').replace('[', '').replace(']', '').replace('Âà†', '').replace('=', '').replace("'", '').replace('/', '').replace("‚ö°", " ").replace(".", '').replace('üê∞', '').replace("„Äê", '').replace('?', '')
  return data

In [ ]:
def lang(word):
  try:
    language = langid.classify(word)[0]
  except:
    language = None
  return language

In [ ]:
df = pd.DataFrame()
#  working with half of the data as the dataset is too large
df['en'] = dataset['train']['english'][:int(len(dataset['train'])/2.5)]
df['mr'] = dataset['train']['marathi'][:int(len(dataset['train'])/2.5)]

df1 = pd.DataFrame()
df1['en'] = dataset['test']['english'][:int(len(dataset['test'])/2.5)]
df1['mr'] = dataset['test']['marathi'][:int(len(dataset['test'])/2.5)]

In [ ]:
def process(df):
  # removing special characters from english text data
  df['en_new'] = list(map(lambda x: filter(x) , df['en']))
  df['mr_new'] = list(map(lambda x: x , df['mr']))
  # drop the previous english and marathi columns
  df.drop(['en', 'mr'], axis = 1, inplace = True)
  # extracting the first 5 letters for langauge detection( masking it a bit faster)
  df['en_wrd'] = list(map(lambda x: x[:5] , df['en_new']))
  df['mr_wrd'] = list(map(lambda x: x[:5] , df['mr_new']))
  # performing language detection using langid function made above
  return df

def filter_lang(df):
  df['inp_lang'] = df['en_wrd'].map(lang)
  df['out_lang']  = df['mr_wrd'].map(lang)
  # dropping coulmns in no language is detected
  df.dropna(subset = ['inp_lang', 'out_lang'], inplace = True)
  # dropping other languages if present in the dataset
  if df['inp_lang'].value_counts()['en'] != len(df['inp_lang']):
    df.drop(df[df['inp_lang'] != 'en'].index, inplace = True)
  # Not do language detection for maratahi as it can be confused with hindi
  df.reset_index(inplace = True)
  df.drop(['en_wrd', 'mr_wrd', 'index', 'inp_lang'm 'out_lang'], axis = 1, inplace = True)
  # dropping duplicate line to make dataset more effective
  df.drop_duplicates(inplace = True)
  # finding out the length of english data
  df['en_len'] = df['en_new'].apply(lambda row: len(row.split(' ')))
  return df

In [ ]:
df_1 = process(df)
df_train = filter_lang(df_1)

In [ ]:
print(len(df_train))
df_train.head(5)

608385


en_new  \
0          Next few months are really crucial for us   
1  Kanwal Singh Chauhan, the president of Sonepat...   
2  He has won several competitions at the state a...   
3                        This is a worrying increase   
4                    Most of the victims were Dalits   

                                              mr_new  en_len  
0  ‡§™‡•Å‡§¢‡•Ä‡§≤ ‡§§‡•Ä‡§® ‡§Æ‡§π‡§ø‡§®‡•á ‡§Ü‡§Æ‡§ö‡•ç‡§Ø‡§æ‡§∏‡§æ‡§†‡•Ä ‡§ñ‡•Ç‡§™ ‡§Æ‡§π‡§§‡•ç‡§§‡•ç‡§µ‡§™‡•Ç‡§∞‡•ç‡§£ ‡§Ü‡§π‡•á‡§§.       8  
1  ‡§µ‡•É‡§§‡•ç‡§§‡§∏‡§Ç‡§∏‡•ç‡§•‡•á‡§®‡•á ‡§¶‡§ø‡§≤‡•á‡§≤‡•ç‡§Ø‡§æ ‡§Æ‡§æ‡§π‡§ø‡§§‡•Ä‡§®‡•Å‡§∏‡§æ‡§∞, ‡§µ‡§ø‡§∞‡•ã‡§ß ‡§ï‡§∞‡§£‡•ç...      15  
2  ‡§∞‡§æ‡§ú‡•ç‡§Ø ‡§µ ‡§∞‡§æ‡§∑‡•ç‡§ü‡•ç‡§∞‡•Ä‡§Ø ‡§™‡§æ‡§§‡§≥‡•Ä‡§µ‡§∞‡•Ä‡§≤ ‡§Ö‡§®‡•á‡§ï ‡§Æ‡§≤‡•ç‡§≤ ‡§§‡•ç‡§Ø‡§æ‡§Ç‡§®‡•Ä ...      11  
3                           ‡§π‡•Ä ‡§µ‡§æ‡§¢ ‡§ö‡§ø‡§Ç‡§§‡§æ‡§ú‡§®‡§ï ‡§§‡§∞ ‡§Ü‡§π‡•á‡§ö.       5  
4  ‡§§‡•ç‡§Ø‡§æ‡§§‡•Ä‡§≤ ‡§¨‡§π‡•Å‡§§‡§æ‡§Ç‡§∂‡•Ä ‡§ò‡§ü‡§®‡§æ‡§Ç‡§§ ‡§™‡•Ä‡§°‡§ø‡§§ ‡§µ‡•ç‡§Ø‡§ï‡•ç‡§§‡•Ä ‡§Ø‡§æ ‡§¶‡§≤‡§ø‡§§ ...       6

In [ ]:
df_2 = process(df1)
df_test = filter_lang(df_2)

In [ ]:
df_test.head(4)

en_new  \
0  This name is derived from two Sanskrit words K...   
1  Speaking on the occasion, Minister of State of...   
2          Two accused of raping minor girl arrested   
3  There was a man of the hill country of Ephraim...   

                                              mr_new  en_len  
0  ‡§π‡•á ‡§®‡§æ‡§µ ‡§¶‡•ã‡§® ‡§∏‡§Ç‡§∏‡•ç‡§ï‡•É‡§§ ‡§∂‡§¨‡•ç‡§¶‡§æ‡§Ç‡§ï‡§°‡•Ç‡§® ‡§™‡•ç‡§∞‡§æ‡§™‡•ç‡§§ ‡§ù‡§æ‡§≤‡•á ‡§Ü‡§π‡•á...      11  
1  ‡§∞‡•á‡§≤‡•ç‡§µ‡•á ‡§∞‡§æ‡§ú‡•ç‡§Ø‡§Æ‡§Ç‡§§‡•ç‡§∞‡•Ä ‡§∏‡•Å‡§∞‡•á‡§∂ ‡§Ö‡§Ç‡§ó‡§°‡•Ä, ‡§≠‡§æ‡§∞‡§§‡•Ä‡§Ø ‡§â‡§¶‡•ç‡§Ø‡•ã‡§ó ...      21  
2  ‡§Ö‡§≤‡•ç‡§™‡§µ‡§Ø‡•Ä‡§® ‡§Æ‡•Å‡§≤‡•Ä‡§µ‡§∞ ‡§Ö‡§§‡•ç‡§Ø‡§æ‡§ö‡§æ‡§∞ ‡§ï‡§∞‡§£‡§æ‡§±‡•ç‡§Ø‡§æ ‡§¶‡•ã‡§® ‡§®‡§∞‡§æ‡§ß‡§Æ‡§æ‡§Ç‡§®...       7  
3  ‡§è‡§´‡•ç‡§∞‡§æ‡§á‡§Æ‡§ö‡•ç‡§Ø‡§æ ‡§°‡•ã‡§Ç‡§ó‡§∞‡§æ‡§≥ ‡§™‡•ç‡§∞‡§¶‡•á‡§∂‡§æ‡§§ ‡§Æ‡•Ä‡§ñ‡§æ ‡§®‡§æ‡§Æ‡§ï ‡§è‡§ï ‡§Æ‡§æ‡§£‡•Ç...      14

In [ ]:
len(df_test)

327569

In [ ]:
# combining the two datasets
frames  = [df_train, df_test]
df_final = pd.concat(frames)
df_final.head(10)

en_new  \
0          Next few months are really crucial for us   
1  Kanwal Singh Chauhan, the president of Sonepat...   
2  He has won several competitions at the state a...   
3                        This is a worrying increase   
4                    Most of the victims were Dalits   
5                      This has shocked the Congress   
6   The funds collected are used for social purposes   
7  Similarly, there is a distance restriction for...   
8  She said, I needed to have my counsellor on th...   
9                                      Dont remember   

                                              mr_new  en_len  
0  ‡§™‡•Å‡§¢‡•Ä‡§≤ ‡§§‡•Ä‡§® ‡§Æ‡§π‡§ø‡§®‡•á ‡§Ü‡§Æ‡§ö‡•ç‡§Ø‡§æ‡§∏‡§æ‡§†‡•Ä ‡§ñ‡•Ç‡§™ ‡§Æ‡§π‡§§‡•ç‡§§‡•ç‡§µ‡§™‡•Ç‡§∞‡•ç‡§£ ‡§Ü‡§π‡•á‡§§.       8  
1  ‡§µ‡•É‡§§‡•ç‡§§‡§∏‡§Ç‡§∏‡•ç‡§•‡•á‡§®‡•á ‡§¶‡§ø‡§≤‡•á‡§≤‡•ç‡§Ø‡§æ ‡§Æ‡§æ‡§π‡§ø‡§§‡•Ä‡§®‡•Å‡§∏‡§æ‡§∞, ‡§µ‡§ø‡§∞‡•ã‡§ß ‡§ï‡§∞‡§£‡•ç...      15  
2  ‡§∞‡§æ‡§ú‡•ç‡§Ø ‡§µ ‡§∞‡§æ‡§∑‡•ç‡§ü‡•ç‡§∞‡•Ä‡§Ø ‡§™‡§æ‡§§‡§≥‡•Ä‡§µ‡§∞‡•Ä‡§≤ ‡§Ö‡§®‡•á‡§ï ‡§Æ‡§≤‡•ç‡§≤ ‡§§‡•ç‡§Ø‡§æ‡§Ç‡§®‡•Ä ...      11  
3                           ‡§π‡•Ä ‡§µ‡§æ‡§¢ ‡§ö‡§ø‡§Ç‡§§‡§æ‡§ú‡§®‡§ï ‡§§‡§∞ ‡§Ü‡§π‡•á‡§ö.       5  
4  ‡§§‡•ç‡§Ø‡§æ‡§§‡•Ä‡§≤ ‡§¨‡§π‡•Å‡§§‡§æ‡§Ç‡§∂‡•Ä ‡§ò‡§ü‡§®‡§æ‡§Ç‡§§ ‡§™‡•Ä‡§°‡§ø‡§§ ‡§µ‡•ç‡§Ø‡§ï‡•ç‡§§‡•Ä ‡§Ø‡§æ ‡§¶‡§≤‡§ø‡§§ ...       6  
5               ‡§§‡•ç‡§Ø‡§æ‡§Æ‡•Å‡§≥‡•á ‡§ï‡§æ‡§Å‡§ó‡•ç‡§∞‡•á‡§∏‡§≤‡§æ ‡§Æ‡•ã‡§†‡§æ ‡§ß‡§ï‡•ç‡§ï‡§æ ‡§¨‡§∏‡§≤‡§æ.       5  
6          ‡§§‡•ç‡§Ø‡§æ ‡§®‡§ø‡§ß‡•Ä‡§ö‡§æ ‡§µ‡§æ‡§™‡§∞ ‡§∏‡§æ‡§Æ‡§æ‡§ú‡§ø‡§ï ‡§ï‡§æ‡§∞‡•ç‡§Ø‡§æ‡§∏‡§æ‡§†‡•Ä ‡§ï‡§∞‡§§‡•ã.       8  
7  ‡§§‡•ç‡§Ø‡§æ‡§ö‡§™‡•ç‡§∞‡§Æ‡§æ‡§£‡•á ‡§∞‡•Ö‡§ï‡•ç‡§∏ ‡§∏‡§æ‡§†‡•Ä ‡§≤‡•ã‡§°‡•Ä‡§Ç‡§ó ‡§™‡•â‡§à‡§Ç‡§ü‡•ç‡§∏ ‡§Æ‡•ç‡§π‡§£‡§ú‡•á ...      34  
8  '‡§ö‡§ø‡§§‡•ç‡§∞‡§™‡§ü‡§æ‡§ö‡•ç‡§Ø‡§æ ‡§∏‡•á‡§ü‡§µ‡§∞ ‡§Æ‡§≤‡§æ ‡§Æ‡§æ‡§ù‡•ç‡§Ø‡§æ ‡§∏‡§Æ‡•Å‡§™‡§¶‡•á‡§∂‡§ï‡§æ‡§Ç‡§ö‡•Ä ‡§ó‡§∞...      16  
9                                         ‡§Ü‡§†‡§µ‡§≤‡•á, ‡§®‡§æ.       2

In [ ]:
len(df_final)

935954

In [ ]:
print('Minimum:', df_final['en_len'].min(), 'Maximum: ' , df_final['en_len'].max(), 'Mean: ', df_final['en_len'].mean(), 'Mode:' , statistics.mode(df_final['en_len']))

Minimum: 1 Maximum:  2411 Mean:  10.481850603768988 Mode: 5


In [ ]:
# so we will be lossing this many data points
count = 0
for i in range(4):
  if i in list(df_final['en_len']):
    count = count + df_final['en_len'].value_counts()[i]

print(count)

81673


In [ ]:
df_final.drop(df_final[df_final['en_len'] <= 4].index , inplace =True, axis = 0)
df_final.reset_index(inplace = True)
df_final.drop(['index', 'en_len'], axis = 1, inplace = True)

In [ ]:
len(df_final)

671781

Observing the length after filtering the sentences

In [ ]:
df_final['en_len'] = df_final['en_new'].apply(lambda row: len(row.split(' ')))
print('Minimum:', df_final['en_len'].min(), 'Maximum: ' , df_final['en_len'].max(), 'Mean: ', df_final['en_len'].mean(), 'Mode:' , statistics.mode(df_final['en_len']))
df_final.drop(['en_len'], axis = 1, inplace = True)

Minimum: 5 Maximum:  2411 Mean:  12.03523767418251 Mode: 5


In [ ]:
split = int(0.80*len(df_final))

df_train = df_final[:split]
df_test = df_final[split:]

In [ ]:
df_train = df_train.sample(frac = 1)
df_train.reset_index(inplace = True)
df_test = df_test.sample(frac = 1)
df_test.reset_index(inplace= True)

In [ ]:
df_train.drop(['index'], inplace = True, axis = 1)
df_test.drop(['index'], inplace = True, axis = 1)

In [7]:
df_train.to_csv('train.csv' , index = False , encoding = 'utf-8')
df_test.to_csv('test.csv' , index = False , encoding = 'utf-8')